In [1]:
import spacy
import re
import csv
from datetime import datetime, date, time
from spacy.tokens import Span
from spacy import displacy

In [2]:
from cut_pp import cut_pp
from cut_noun_phrases import cut_noun_phrases
from make_span_list import make_span_list
from define_structure import define_structure
from case_for_simplified_structure import case_for_simplified_structure
from find_case_final import find_case_final

In [3]:
nlp = spacy.load("de_core_news_lg")

In [4]:
filename = "input_files/wegen_dereko_1990-2020.txt"
file = open(filename, "r").read()

In [5]:
korpus = "dereko_1990-2020"

In [9]:
text_splitted = file.split("\n")

19999
Was tun? Ihn beim Ballverteilen zu stören, ist wegen der Technik des 33-Jährigen eine schwierige Aufgabe. Klüger ist es deshalb, die Adressaten von Ze Robertos Pässen eng zu decken. Dann muss er selbst aufs Tor zusteuern, das er immer noch eher selten trifft. (HAZ07/AUG.01851 Hannoversche Allgemeine, 25.08.2007, S. 27; Gegenmittel gesucht)


In [10]:
dirname = "output_files/"
prep = "wegen"
name_csv = dirname + prep + '_' + korpus + "_" + str(datetime.today())[:10] +'.csv'
row = ["whole span", "phrase" , "case_final", "structure" , "head", "noun", "remark", "flag_von", "number", "gender", "source" , "paragraph"]

['whole span', 'phrase', 'case_final', 'structure', 'head', 'noun', 'remark', 'flag_von', 'number', 'gender', 'source', 'paragraph']


In [11]:
docs = list(nlp.pipe(text_splitted))

CPU times: user 1min 19s, sys: 19 s, total: 1min 38s
Wall time: 1min 38s


In [13]:
%%time
prep = "wegen"
with open(name_csv, 'w') as f:
    writer = csv.writer(f, delimiter='#')
    writer.writerow(row)    
    for doc in docs:
        if doc: 
            quelle = doc.text.split("(")[-1]
            quelle_short = re.findall(r"([A-Z]{1,3})[0-9]{,2}/", quelle)
            flag_von = 0
            find_von = re.findall("[Vv]on wegen", doc.text) 
            if len(find_von) > 0:
                flag_von = 1
                continue
            num = 0
            for token in doc:
                if token.lemma_ == prep:
                    span =  doc[token.left_edge.i : token.right_edge.i+1]
                    head = token.head.lemma_
                    phrase = span.text
                    
                    if len(phrase) > 0:
                        remark = ""
                        # сбор информации про структуру спана
                        nouns, adjs, articles, short_span, structure, remark, number, gender, lemma_vs_token = define_structure(span, remark, prep)
                        find_case_final(structure, nouns, adjs, articles, short_span, doc, prep, writer, head, remark, number, gender, lemma_vs_token, flag_von, quelle_short)
                    i += 1